In [1]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide root window
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()  # Properly destroy the tkinter root window after selection
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()  # Properly destroy root window after getting input
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\ Gift Card.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\ Credit Card.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\ Cash.xlsx


In [3]:
# import tkinter as tk
# from tkinter import filedialog, messagebox, simpledialog, ttk
# temp_root = tk.Tk()
# temp_root.withdraw()
# messagebox.showerror(
#     "Error", 
#     "❌ You selected all sheets to exclude, including the one you want to split.\n\n"
#     "Exiting... Please re-run the tool with proper selections.",
#     parent=temp_root
# )
# temp_root.destroy()

In [1]:
# 22-4-2025
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"
def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")


def show_error_gracefully(title, message):
    temp_root = tk.Tk()
    temp_root.withdraw()
    messagebox.showerror(title, message, parent=temp_root)
    temp_root.destroy()


    
def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide root window
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()  # Properly destroy the tkinter root window after selection
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()  # Properly destroy root window after getting input
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}",
            parent=temp_root
            )
        temp_root.destroy()
        
        
        if sheet_to_split in exclude_sheets:
            temp_root = tk.Tk()
            temp_root.withdraw()
#             messagebox.showinfo("Error", "excluded all sheets.")
            print("exiting")

#         if tk._default_root:
# #             for widget in tk._default_root.winfo_children():
# #                 widget.destroy()
# #             tk._default_root.destroy()
            messagebox.showerror("Error", 
                "❌ You selected all sheets to exclude, including the one you want to split.\n\n"
                "Exiting... Please re-run the tool with proper selections.")

            if tk._default_root:
                for widget in tk._default_root.winfo_children():
                    widget.destroy()
                tk._default_root.destroy()
            return


        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        print(output_dir)
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}",)
        print("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Error", "excluded all sheets.",e)
        print("Exiting....")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


exiting


In [5]:
# final run
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()  # ✅ Ensure window is closed

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()  # ✅ Fully close the window
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Cleanup after loop ends
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Fully close after use
    return result

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Set as the only selected tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Clear previous data except header
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Write filtered data
                filtered_df = df[df[split_column] == val]
                for i, row in enumerate(filtered_df.values, start=2):
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=i, column=j).value = value

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
#                 messagebox.showinfo("Done", f"Files split and formatting preserved!\nSaved to:\n{output_dir}")

        # Force close any leftover windows
        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\1.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\2.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\3.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\4.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\5.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\6.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\7.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\8.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\9.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\10.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\11.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\12.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\13.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\14.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\15.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\16.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\17.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\18.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\19.xlsx


KeyboardInterrupt: 

In [1]:
# format preservation
# final run with formatting & formula preservation
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from copy import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

# 👉 Helper to copy formatting & formula
def copy_row(ws_from, ws_to, row_idx_from, row_idx_to):
    for col_idx, cell in enumerate(ws_from[row_idx_from], start=1):
        new_cell = ws_to.cell(row=row_idx_to, column=col_idx)
        new_cell.value = cell.value
        new_cell.data_type = cell.data_type

        if cell.has_style:
            new_cell.font = copy(cell.font)
            new_cell.border = copy(cell.border)
            new_cell.fill = copy(cell.fill)
            new_cell.number_format = copy(cell.number_format)
            new_cell.protection = copy(cell.protection)
            new_cell.alignment = copy(cell.alignment)

        if cell.hyperlink:
            new_cell.hyperlink = copy(cell.hyperlink)

        if cell.comment:
            new_cell.comment = copy(cell.comment)

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            if sheet_to_split not in wb.sheetnames:
                continue

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            ws = wb[sheet_to_split]

            # Set as active tab
            for w in wb.worksheets:
                w.sheet_view.tabSelected = False
            ws.sheet_view.tabSelected = True
            wb.active = wb.sheetnames.index(sheet_to_split)

            # Copy original sheet (used to preserve formulas/formatting)
            original_ws = load_workbook(file_path, data_only=False)[sheet_to_split]

            # Clear old data below header
            max_row = ws.max_row
            if max_row > 1:
                ws.delete_rows(2, max_row - 1)

            filtered_df = df[df[split_column] == val]

            for idx, (_, row) in enumerate(filtered_df.iterrows(), start=2):
                source_excel_row_index = row.name + 2  # Adjust for Excel's row numbers
                copy_row(original_ws, ws, source_excel_row_index, idx)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


UnicodeEncodeError: 'charmap' codec can't encode character '\u274c' in position 0: character maps to <undefined>

In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.worksheet import Worksheet

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def copy_row(source_ws: Worksheet, dest_ws: Worksheet, source_row: int, dest_row: int):
    for col in range(1, source_ws.max_column + 1):
        source_cell = source_ws.cell(row=source_row, column=col)
        dest_cell = dest_ws.cell(row=dest_row, column=col)

        dest_cell.value = source_cell.value
        dest_cell.data_type = source_cell.data_type
        dest_cell.number_format = source_cell.number_format
        dest_cell.font = source_cell.font
        dest_cell.border = source_cell.border
        dest_cell.fill = source_cell.fill
        dest_cell.alignment = source_cell.alignment

        if source_cell.has_formula:
            dest_cell.value = f"={source_cell.value}"

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            original_wb = load_workbook(file_path)
            original_ws = original_wb[sheet_to_split]

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Set tab active
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Clear previous data except header
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Filtered data
                filtered_df = df[df[split_column] == val]

                # ✅ Copy matching rows with formatting and formulas
                for i in range(len(filtered_df)):
                    excel_source_row_index = filtered_df.index[i] + 2  # original Excel row (1-based + header)
                    target_row_index = i + 2  # destination row in new sheet
                    copy_row(original_ws, ws, excel_source_row_index, target_row_index)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        # Close any leftover windows
        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xlsm")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")
    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_everything():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, dtype=str)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            if sheet_to_split not in wb.sheetnames:
                continue

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            ws = wb[sheet_to_split]
            headers = [cell.value for cell in ws[1]]

            # Create a mapping of column name to column number
            col_map = {cell.value: idx+1 for idx, cell in enumerate(ws[1])}
            filter_col_idx = col_map.get(split_column)
            if not filter_col_idx:
                continue

            rows_to_keep = [1]  # Always keep header
            for i in range(2, ws.max_row + 1):
                cell_val = ws.cell(i, filter_col_idx).value
                if str(cell_val).strip() == str(val).strip():
                    rows_to_keep.append(i)

            # Delete rows not in the filtered list
            all_rows = list(range(2, ws.max_row + 1))
            delete_rows = [i for i in all_rows if i not in rows_to_keep]

            for row in reversed(delete_rows):
                ws.delete_rows(row, 1)

            # Ensure it's the active selected sheet
            for w in wb.worksheets:
                w.sheet_view.tabSelected = False
            ws.sheet_view.tabSelected = True
            wb.active = wb.sheetnames.index(sheet_to_split)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split successfully!\nSaved to:\n{output_dir}")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_everything()


In [ ]:
#  Save formulas too
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Select this tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Backup formulas from first data row
                start_row = 2
                formulas = []
                for cell in ws[start_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                # Clear previous data rows
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Filter data
                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    # Apply formulas row-wise
                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
# Final Code with formatting 

import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from tkinter import simpledialog  # Add this if you need askstring
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=1"0)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")
    root.attributes("-topmost", True)

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    root.attributes("-topmost", True)

    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()
    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", msg)
        log_error(msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                start_row = 2
                formulas = []
                for cell in ws[start_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split with formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", msg)
        log_error(msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
# perfect just adjust data start row

import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
from copy import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=10)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Select this tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Backup template row 2
                start_row = 2
                template_row = list(ws.iter_rows(min_row=start_row, max_row=start_row))[0]
                template_data = []
                for cell in template_row:
                    template_data.append({
                        "value": cell.value,
                        "is_formula": cell.data_type == "f",
                        "style": copy(cell._style),
                        "number_format": cell.number_format
                    })

                # Clear all data rows after header
                if ws.max_row > 1:
                    ws.delete_rows(start_row, ws.max_row - 1)

                # Filter data
                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, (value, cell_template) in enumerate(zip(row, template_data), start=1):
                        cell = ws.cell(row=start_row + i, column=j)
                        cell.value = value
                        cell._style = copy(cell_template["style"])
                        cell.number_format = cell_template["number_format"]

                    for j, cell_template in enumerate(template_data, start=1):
                        if cell_template["is_formula"]:
                            origin = f"{get_column_letter(j)}{start_row}"
                            target = f"{get_column_letter(j)}{start_row + i}"
                            translated_formula = Translator("=" + str(cell_template["value"]), origin=origin).translate_formula(target)
                            cell = ws.cell(row=start_row + i, column=j)
                            cell.value = translated_formula
                            cell._style = copy(cell_template["style"])
                            cell.number_format = cell_template["number_format"]

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split with formatting + formulas!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x250")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=10)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x200")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def ask_start_row():
    result = {"value": 1}

    def confirm():
        try:
            row_val = int(entry.get())
            if row_val >= 1:
                result["value"] = row_val
        except:
            pass
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("Data Start Row")
    root.geometry("300x150")

    ttk.Label(root, text="Enter the row number where data headers start:").pack(pady=10)
    entry = ttk.Entry(root)
    entry.insert(0, "1")
    entry.pack(pady=5)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result["value"]

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = ask_start_row()

        # Use xlwings to fetch header row
        app_temp = xw.App(visible=False)
        wb_temp = app_temp.books.open(file_path)
        ws_temp = wb_temp.sheets[sheet_to_split]
        header_row = ws_temp.range((start_row, 1), (start_row, ws_temp.used_range.last_cell.column)).value
        wb_temp.close()
        app_temp.quit()

        split_column = get_dropdown_selection(header_row, "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=None)
        df.columns = df.iloc[start_row - 1]
        df = df.iloc[start_row:]

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                start_excel_row = start_row + 1
                formulas = []
                for cell in ws[start_excel_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > start_excel_row:
                    ws.delete_rows(start_excel_row + 1, max_row - start_excel_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_excel_row + i, column=j, value=value)

                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_excel_row}"
                            target_cell = f"{get_column_letter(j)}{start_excel_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_excel_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        subprocess.Popen(f'explorer "{output_dir}"')
        messagebox.showinfo("Done", f"🎉 Files split and formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
# 18-4-2025

In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()
    return row

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                for cell in ws[excel_data_start]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)  # Keep header

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i  # Write below header
                        ws.cell(row=cell_row, column=j, value=value)

                        # Reapply formulas if applicable
                        orig_formula = formulas[j - 1]
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=cell_row, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formulas preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
import shutil
import tempfile
import subprocess
import datetime

def log_error(msg):
    print("Error:", msg)

def get_selected_items(listbox, items):
    selected_indices = listbox.curselection()
    return [items[i] for i in selected_indices]

def main():
    root = tk.Tk()
    root.withdraw()

    try:
        # Step 1: Welcome message
        welcome = tk.Toplevel()
        welcome.title("Welcome")
        welcome.geometry("400x150")
        tk.Label(welcome, text="📂 Excel Splitter Tool", font=("Arial", 16, "bold")).pack(pady=10)
        tk.Label(welcome, text="This tool will split your Excel sheet based on selected values.").pack()
        tk.Button(welcome, text="Get Started", command=welcome.destroy).pack(pady=10)
        welcome.grab_set()
        welcome.mainloop()

        # Step 2: Ask user to select file
        file_path = filedialog.askopenfilename(
            title="Select Excel file",
            filetypes=[("Excel Files", "*.xlsx *.xls")]
        )
        if not file_path:
            messagebox.showinfo("No file selected", "Operation cancelled.")
            return

        # Step 3: Load sheet names
        wb = load_workbook(file_path, data_only=True)
        sheet_names = wb.sheetnames

        # Step 4: Sheet selection window
        sheet_win = tk.Toplevel()
        sheet_win.title("Select Sheet")
        sheet_win.geometry("300x150")
        tk.Label(sheet_win, text="Select sheet to split:").pack(pady=5)
        selected_sheet = tk.StringVar(value=sheet_names[0])
        sheet_dropdown = ttk.Combobox(sheet_win, textvariable=selected_sheet, values=sheet_names)
        sheet_dropdown.pack(pady=10)

        def confirm_sheet():
            if selected_sheet.get() not in sheet_names:
                messagebox.showerror("Invalid", "Please select a valid sheet.")
            else:
                sheet_win.destroy()

        tk.Button(sheet_win, text="Next", command=confirm_sheet).pack(pady=10)
        sheet_win.grab_set()
        sheet_win.mainloop()

        # Step 5: Ask for header start row
        header_row = simpledialog.askinteger("Header Row", "Enter row number where header starts:")
        if not header_row:
            messagebox.showerror("Invalid Input", "Header row not specified.")
            return

        # Load selected sheet into DataFrame
        df = pd.read_excel(file_path, sheet_name=selected_sheet.get(), header=header_row - 1)
        column_choices = list(df.columns)

        # Step 6: Ask user to select the column to split by
        col_win = tk.Toplevel()
        col_win.title("Select Split Column")
        col_win.geometry("400x150")
        tk.Label(col_win, text="Select column to split by:").pack(pady=5)
        selected_col = tk.StringVar(value=column_choices[0])
        col_dropdown = ttk.Combobox(col_win, textvariable=selected_col, values=column_choices)
        col_dropdown.pack(pady=10)

        def confirm_col():
            if selected_col.get() not in column_choices:
                messagebox.showerror("Invalid", "Please select a valid column.")
            else:
                col_win.destroy()

        tk.Button(col_win, text="Next", command=confirm_col).pack(pady=10)
        col_win.grab_set()
        col_win.mainloop()

        split_column = selected_col.get()

        # Step 7: Ask user to select which values to split
        unique_vals = df[split_column].dropna().unique().tolist()

        select_win = tk.Toplevel()
        select_win.title("Select Values to Split")
        select_win.geometry("400x400")
        tk.Label(select_win, text="Select values to create individual files:").pack(pady=5)

        listbox = tk.Listbox(select_win, selectmode=tk.MULTIPLE)
        for val in unique_vals:
            listbox.insert(tk.END, str(val))
        listbox.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        selected_vals = []

        def confirm_values():
            selected_vals.extend(get_selected_items(listbox, unique_vals))
            if not selected_vals:
                messagebox.showerror("Error", "Please select at least one value.")
                return
            select_win.destroy()

        tk.Button(select_win, text="Split Files", command=confirm_values).pack(pady=10)
        select_win.grab_set()
        select_win.mainloop()

        if not selected_vals:
            return

        # Step 8: Ask where to save the split files
        output_dir = filedialog.askdirectory(title="Select folder to save split files")
        if not output_dir:
            messagebox.showinfo("No folder", "No folder selected. Exiting.")
            return

        # Step 9: Split and save
        for val in selected_vals:
            temp_dir = tempfile.mkdtemp()
            temp_file = os.path.join(temp_dir, "temp.xlsx")
            shutil.copy(file_path, temp_file)

            wb = load_workbook(temp_file)
            if selected_sheet.get() in wb.sheetnames:
                ws = wb[selected_sheet.get()]
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(selected_sheet.get())

                col_index = df.columns.get_loc(split_column) + 1  # 1-based index

                rows_to_delete = []
                for row in range(header_row + 1, ws.max_row + 1):
                    cell_value = ws.cell(row=row, column=col_index).value
                    if cell_value != val:
                        rows_to_delete.append(row)

                for row in reversed(rows_to_delete):
                    ws.delete_rows(row)

                # Reinstate header (if needed)
                for col_num, column_name in enumerate(df.columns, 1):
                    ws.cell(row=header_row, column=col_num, value=column_name)

                safe_val = str(val).replace("/", "_").replace("\\", "_").replace(":", "_")
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                new_file = os.path.join(output_dir, f"{safe_val}_{timestamp}.xlsx")
                wb.save(new_file)

            shutil.rmtree(temp_dir)

        # Final message
        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

    finally:
        if tk._default_root:
            try:
                for widget in tk._default_root.winfo_children():
                    widget.destroy()
                tk._default_root.destroy()
            except:
                pass

if __name__ == "__main__":
    main()


In [ ]:
## ok


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 2):", initialvalue=1)
    root.destroy()
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()
    return row

def show_processing_window():
    processing_root = tk.Tk()
    processing_root.title("Processing")
    processing_root.geometry("400x100")
    processing_root.resizable(False, False)
    processing_root.attributes("-topmost", True)
    ttk.Label(processing_root, text="📂 Splitter Tool is working on files...", font=("Segoe UI", 12)).pack(expand=True)
    processing_root.update()
    return processing_root

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        processing_window = show_processing_window()

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                for cell in ws[excel_data_start]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        processing_window.destroy()
        messagebox.showinfo("Done", f"🎉 Files split and formulas preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()
